# AI Feynman 2.0 in Google Colab
このノートブックでは AI Feynman を使ってアップロードしたデータから数式を自動発見します。
パラメータ調整やデータの入出力を扱いやすくするため `ipywidgets` を利用した簡易GUIを用意しています。

In [ ]:
# ライブラリのインストール (必要に応じて)
!pip install feynman

In [ ]:
import io
import tempfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from feynman import run_feynman
import ipywidgets as widgets
from IPython.display import display, clear_output, Latex

In [ ]:
# アップロードとパラメータ入力用ウィジェット
upload = widgets.FileUpload(accept='.csv', multiple=False)
x_select = widgets.SelectMultiple(options=[], description='X columns')
y_select = widgets.Dropdown(options=[], description='Y column')

BF_try_time = widgets.IntText(value=30, description='BF_try_time')
polyfit_deg = widgets.IntText(value=5, description='polyfit_deg')
NN_epochs = widgets.IntText(value=1000, description='NN_epochs')
N_SAMPLES = widgets.IntText(value=5000, description='N_SAMPLES')

run_button = widgets.Button(description='実行')
output = widgets.Output()

def on_upload_change(change):
    if upload.value:
        uploaded = list(upload.value.values())[0]
        df = pd.read_csv(io.BytesIO(uploaded['content']))
        cols = df.columns.tolist()
        x_select.options = cols
        y_select.options = cols
        if len(cols) > 1:
            x_select.value = tuple(cols[:-1])
            y_select.value = cols[-1]
        else:
            x_select.value = tuple()
            y_select.value = cols[0]
upload.observe(on_upload_change, names='value')

display(widgets.VBox([
    widgets.HTML('<b>CSV ファイルをアップロードしてください (最終列が Y として想定されます)</b>'),
    upload,
    x_select,
    y_select,
    BF_try_time,
    polyfit_deg,
    NN_epochs,
    N_SAMPLES,
    run_button,
    output
]))


In [ ]:
def on_run_clicked(b):
    with output:
        clear_output()
        if not upload.value:
            print('CSV ファイルをアップロードしてください')
            return
        uploaded = list(upload.value.values())[0]
        df = pd.read_csv(io.BytesIO(uploaded['content']))
        x_cols = list(x_select.value)
        y_col = y_select.value
        X = df[x_cols].values
        y = df[y_col].values
        with tempfile.NamedTemporaryFile(mode='w+', suffix='.csv', delete=False) as tmp:
            full_data = pd.concat([df[x_cols], df[[y_col]]], axis=1)
            full_data.to_csv(tmp.name, index=False, header=False)
            fname = tmp.name
        varnames = x_cols + [y_col]
        results = run_feynman(
            fname,
            varnames=varnames,
            BF_try_time=BF_try_time.value,
            polyfit_deg=polyfit_deg.value,
            NN_epochs=NN_epochs.value,
            N_SAMPLES=N_SAMPLES.value
        )
        print('AI Feynman による最良数式:')
        display(Latex(results['best_eq']))
        print('詳細結果:')
        print(results)
        y_pred = results['best_func'](X.T)
        plt.figure()
        plt.scatter(y, y_pred, alpha=0.5)
        plt.xlabel('実測値')
        plt.ylabel('予測値')
        plt.title('実測値 vs 予測値 (AI Feynman)')
        plt.show()

run_button.on_click(on_run_clicked)
